In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import datetime


In [2]:
from dkvs.bptree import BPTree 

**Даты в обратном порядке**

In [3]:
class Key(msgspec.Struct, frozen=True, order=True):
    
    k: int
    ts: datetime.datetime
    
    def __lt__(self, other: 'Key') -> bool:
        if self.k < other.k:
            return True
        if self.k == other.k:
            if self.ts > other.ts:   # reverse order
                return True
        return False

    def __gt__(self, other) -> bool:
        if other < self:
            return True
        return other < self

    def __le__(self, other) -> bool:
        if (self < other) or (self == other):
            return True
        return False

    def __ge__(self, other) -> bool:
        if (self > other) or (self == other):
            return True
        return False


class Data(msgspec.Struct):
    data: Any | None = None


In [4]:
k1 = Key(1, datetime.datetime.fromisoformat("2012-09-14"))
k2 = Key(1, datetime.datetime.fromisoformat("2000-09-14"))
k3 = Key(1, datetime.datetime.fromisoformat("2000-09-14"))


k1 < k2, k1 > k2, k2 < k1, k2 > k1, k2 == k1, k1 == k2, k2 == k3, k3 == k2

(True, False, False, True, False, False, True, True)

In [5]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")
index = BPTree[Key, Data]()

for _ in range(50):
    index.insert(
        Key(fake.random.randint(0, 9), datetime.datetime.fromisoformat("2024-01-01") - datetime.timedelta(hours = fake.random.randint(0, 100000))),
        Data(fake.name())
    )

In [6]:
for i, r in enumerate(index.keys):
    print(i, r, index.values[i])

0 Key(k=2, ts=datetime.datetime(2015, 6, 28, 10, 0)) Data(data='Ангелина Ждановна Ковалева')
1 Key(k=7, ts=datetime.datetime(2012, 8, 15, 4, 0)) Data(data='Натан Глебович Белов')
2 Key(k=7, ts=datetime.datetime(2023, 7, 30, 5, 0)) Data(data='Мария Оскаровна Кошелева')
3 Key(k=0, ts=datetime.datetime(2013, 8, 5, 20, 0)) Data(data='Иван Бориславович Мясников')
4 Key(k=5, ts=datetime.datetime(2023, 7, 17, 23, 0)) Data(data='Кулагин Тит Августович')
5 Key(k=6, ts=datetime.datetime(2013, 9, 25, 22, 0)) Data(data='Родионов Алексей Гурьевич')
6 Key(k=7, ts=datetime.datetime(2016, 8, 2, 5, 0)) Data(data='Коновалова Елизавета Дмитриевна')
7 Key(k=7, ts=datetime.datetime(2019, 9, 1, 10, 0)) Data(data='Наина Федоровна Стрелкова')
8 Key(k=1, ts=datetime.datetime(2021, 3, 21, 17, 0)) Data(data='Владимирова Кира Васильевна')
9 Key(k=5, ts=datetime.datetime(2013, 3, 18, 18, 0)) Data(data='Прасковья Олеговна Селезнева')
10 Key(k=3, ts=datetime.datetime(2019, 6, 19, 5, 0)) Data(data='Сигизмунд Юльевич 

In [7]:
index.validate()

{}

In [8]:
index.print_node(index.tree[9])


node-idx:9, prev:4, next:11, level: 0 
|-> Key(k=7, ts=datetime.datetime(2012, 8, 15, 4, 0))
|-> Key(k=8, ts=datetime.datetime(2022, 1, 25, 4, 0))
|-> Key(k=8, ts=datetime.datetime(2020, 6, 28, 14, 0))


In [9]:
index.find(Key(k=8, ts=datetime.datetime(2022, 1, 25, 4, 0)))

(Data(data='Шилова Евфросиния Олеговна'),
 LeafNode(node_index=9, level=0, prev_node=4, next_node=11, pointers=array([ 1, 20, 47], dtype=int32)),
 20)

In [10]:
index.values[20]

Data(data='Шилова Евфросиния Олеговна')

In [11]:
index.delete(Key(k=8, ts=datetime.datetime(2022, 1, 25, 4, 0)))

True

In [12]:
index.find(Key(k=8, ts=datetime.datetime(2022, 1, 25, 4, 0)))

(None,
 LeafNode(node_index=9, level=0, prev_node=4, next_node=11, pointers=array([ 1, 20, 47], dtype=int32)),
 20)

In [13]:
index.values[20]

In [14]:
index.delete(Key(k=8, ts=datetime.datetime(2022, 1, 25, 4, 0)))

False

In [15]:
index.find(Key(k=9, ts=datetime.datetime(2023, 6, 26, 11, 0)))

(Data(data='Олимпий Фёдорович Харитонов'),
 LeafNode(node_index=15, level=0, prev_node=11, next_node=None, pointers=array([11, 32, 40], dtype=int32)),
 11)

In [16]:
index.find(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)))

(Data(data='Акулина Дмитриевна Соколова'),
 LeafNode(node_index=15, level=0, prev_node=11, next_node=None, pointers=array([11, 32, 40], dtype=int32)),
 32)

In [17]:
index.values[32]

Data(data='Акулина Дмитриевна Соколова')

In [18]:
index.delete(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)))

True

In [19]:
index.validate()

{}

In [20]:
index.delete(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)))

False

In [21]:
index.insert(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)), Data("REPLACED"))

(32, None)

In [22]:
index.insert(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)), Data("REPLACED"))

(None, 32)

In [23]:
index.values[32]

Data(data='REPLACED')

In [24]:
index.delete(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)))

True

In [25]:
index.insert(Key(k=9, ts=datetime.datetime(2017, 7, 18, 15, 0)), Data( "REPLACED: " + fake.name()))

(32, None)

In [26]:
index.values[32]

Data(data='REPLACED: Кузьма Данилович Беспалов')

In [27]:
index.validate()

{}